In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
%cd 'drive/My Drive/2516Project'
!ls

/content/drive/My Drive/2516Project
data  elmo  glove  result  saved_models  saved_models_cnn


In [0]:
import pandas as pd 
import string
import numpy as np
import xml.etree.ElementTree as ET
import re
import sys
import nltk
from nltk.corpus import stopwords
from urllib.parse import urlparse
import argparse
import os
import time
from html.parser import HTMLParser
from nltk.corpus import stopwords as NLTK_STOPWORDS
from gensim.parsing.preprocessing import STOPWORDS as GENSIM_STOPWORDS
import tldextract
from collections import Counter
import json
from numbers import Number


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
!pip install tldextract

In [0]:
!pip install allennlp

     |████████████████████████████████| 7.6MB 3.1MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 5.1MB 40.9MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 256kB 48.0MB/s 
     |████████████████████████████████| 133kB 48.6MB/s 
     |████████████████████████████████| 245kB 47.7MB/s 
     |████████████████████████████████| 163kB 49.1MB/s 
     |████████████████████████████████| 204kB 50.6MB/s 
     |████████████████████████████████| 30.9MB 144kB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 1.0MB 32.9MB/s 
     |████████████████████████████████| 2.1MB 35.1MB/s 
     |████████████████████████████████| 92kB 10.3MB/s 
     |████████████████████████████████| 3.2MB 39.0MB/s 
  Created wheel for parsimonious: filename=parsimonious-0.8.1-cp36-none-any.whl size=42712 sha256=947ab432ca03ba03737d906f5e7c86726d65471eaac61178a8f8fb375a6aba60
  Sto

# Parsing HTML

In [0]:
def clean_text(text):
    text = text.replace("”", "\"")
    text = text.replace("“", "\"")
    text = text.replace("’", "'")
    text = text.replace("&amp;", "&")
    text = text.replace(" _", " ")
    text = text.replace("–", "-")
    text = text.replace("&lt;", "<")
    text = text.replace("&gt;", ">")
    text = text.replace("<p>", " ")
    text = text.replace("</p>", " ")
    text_to_space = re.compile('&#160;')
    text = text_to_space.subn(' ', text)[0]
    return text

class MyHTMLParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.ignore = False
        self.data = []
        self.temp = []
        self.type1 = ['script', 'style']
        self.type2 = ['p', 'br']

    def append_paragraph(self):
        if len(self.temp) > 0:
            self.data.append(self.temp)
            self.temp = []

    def handle_starttag(self, tag, attrs):
        if tag in self.type1:
            self.ignore = True
        elif tag in self.type2:
            self.append_paragraph()

    def handle_endtag(self, tag):
        if tag in self.type1:
            self.ignore = False
        elif tag in self.type2:
            self.append_paragraph()

    def handle_startendtag(self, tag, attrs):
        if tag in self.type2:
            self.append_paragraph()

    def handle_data(self, data):
        if not self.ignore:
            self.temp.append(data)

    def close(self):
        HTMLParser.close(self)
        self.append_paragraph()

    def reset(self):
        HTMLParser.reset(self)
        self.data = []
        self.temp = []

    def clean_para(self, text):
        text = clean_text(text)
        text = text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
        text = ' '.join(text.split()).strip()
        return text

    def paragraphs(self):
        pars = []
        for par in self.data:
            if len(par) > 0:
                text = self.clean_para(''.join(par)).strip()
                if text:
                    pars.append(text)
        return pars


# Preprocessing 

In [0]:
stopwords = set(NLTK_STOPWORDS.words('english'))
for w in GENSIM_STOPWORDS:
    stopwords.add(w)
stopwords.add("'s")
stopword_list = list(stopwords)
for s in stopword_list:
    stopwords.add(s.capitalize())

In [0]:
class Article_to_Line:
    def __init__(self, stream, features):
        self.stream = stream
        self.features = features 

    def extract_features(self, doc, features):
        result = []
        for f in features:
            val = doc.get(f)
            result.append(val)
        return result

    def __call__(self, article):
        vals = self.extract_features(article, self.features)
        strings = []
        for i in range(len(vals)):
            val = vals[i]
            if isinstance(val, str):
                strings.append(val)
            elif isinstance(val, Number):
                strings.append(str(val))
            elif isinstance(val, list):
                strings.append(json.dumps(val))
            elif isinstance(val, dict):
                strings.append(json.dumps(val))
        print(article['id'], article.get('target'),
                article.get('bias'), article.get('domain'),
                "\t".join(strings), file=self.stream, sep="\t")


class AddTarget:
    def __init__(self, target_, bias_, url_):
        self.target_ = target_
        self.bias_ = bias_
        self.url_ = url_
    def __call__(self, article):
        id = article['id']
        target = self.target_[id]
        bias = self.bias_[id]
        url = self.url_[id]
        article['target'] = target
        article['bias'] = bias
        article['url'] = url


class AddTitle:
    def __call__(self, article):
        article['title'] = clean_text(article['et'].attrib["title"])


class AddText:
    def __init__(self):
        self.parser = None 
    
    def __call__(self, article):
        if self.parser is None:
            self.parser = MyHTMLParser()
        self.parser.reset()
        self.parser.feed(article['xml'])
        self.parser.close()
        pars = self.parser.paragraphs()
        article['pars'] = pars
        text = " ".join(pars)
        article['text'] = text


class RemoveParagraphs:
    def __call__(self, article):
        del article['pars']


class Filtered_Text:
    def f_tokens(self, tokens):
        punctuation = '.,:;?!"\'\'``+={}[]()#~$--'
        re_num = re.compile('^-?[0-9.,]+([eE^][0-9]+)?(th)?$')
        filtered = [token for token in tokens if token not in punctuation and "_" not in token and token not in stopwords]
        filtered = [re_num.subn("<num>", token)[0] for token in filtered]
        return filtered
    
    def __call__(self, article):
        text, title = article['text_tokens'], article['title_tokens']
        tokens = self.f_tokens([t[0] for t in title])
        tokens.append("<sep_t2d>")
        if article.get('link_domains_all'):
            tokens.extend(["DOMAIN_" + d for d in article['link_domains']])
        tokens.append("<sep_d2a>")
        tokens.extend(self.f_tokens([t[0] for sent in text for t in sent]))
        token_string = " ".join(tokens)
        article['text_all_filtered'] = token_string


class NlpSpacy:
    def __init__(self):
        self.initialized = False

    def __call__(self, article):
        if not self.initialized:
            self.nlp = spacy.load("en_core_web_sm", disable=["parser"])
            self.nlp.add_pipe(self.nlp.create_pipe('sentencizer'))
            self.initialized = True
        pars = article['pars']
        article['n_p'] = len(pars)
        n_p_filled = 0
        docs = list(self.nlp.pipe(pars))
        allthree = [[[t.text, t.pos_, t.lemma_] for t in s] for doc in docs for s in doc.sents]
        article['n_p_filled'] = n_p_filled
        article['text_tokens'] = allthree
        ents = [ent.text for doc in docs for ent in doc.ents if ent.text[0].isupper()]
        article['text_ents'] = ents
        title = article['title']
        doc = self.nlp(title)
        allthree = [(t.text, t.pos_, t.lemma_) for s in list(doc.sents) for t in s]
        article['title_tokens'] = allthree
        ents = [ent.text for ent in doc.ents if ent.text[0].isupper()]
        article['title_ents'] = ents


class SequenceSentences:
    def __call__(self, article):
        title_sent = " ".join([t[0] for t in article['title_tokens']])
        article['title_sent'] = title_sent
        
        domain_sent = ""
        if article.get('link_domains_all'):
            domain_sent = " ".join(["DOMAIN_" + d for d in article['link_domains']])
        article['domain_sent'] = domain_sent

        all_article = []
        first = True
        for sent in article['text_tokens']:
            if first:
                first = False
            else:
                all_article.append("<splt>")
            for t in sent:
                all_article.append(t[0])

        article_sent = " ".join(all_article)
        article['article_sent'] = article_sent
        


# Convert to TSV

In [0]:
!pip install -U spacy

Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)


In [0]:
import spacy

In [0]:
def process_xml(file, funcs):
    tree = ET.iterparse(file)
    n = 0
    for event, element in tree:
        if element.tag == "article":
            attrs = element.attrib
            id = attrs['id']
            published = attrs.get("published-at")
            xml = ET.tostring(element, encoding="utf-8", method="xml").decode()
            article = {
                'id': id,
                'xml': xml,
                'published-at': published,
                'et': element
            }
            for func in funcs:
                func(article)
            del article['et']
            n += 1

    return n

article_path = 'data/articles-training-byarticle-20181122.xml'
target_path = 'data/ground-truth-training-byarticle-20181122.xml'
out_path = 'data/train.text.tsv'
features = ['article_sent', 'title_sent']

print("loading...")

target, bias, url = {}, {}, {}
tree = ET.parse(target_path)
root = tree.getroot()
for child in root:
    attribs = child.attrib
    id = attribs["id"]
    target[id], bias[id], url[id] = attribs["hyperpartisan"], attribs.get("bias"), attribs["url"]

pipeline = [AddTarget(target, bias, url)]

pipeline.extend([
    AddTitle(),
    AddText(),
    NlpSpacy(),
    Filtered_Text(),
    SequenceSentences(),
    RemoveParagraphs(),
])

with open(out_path, "wt", encoding="utf8") as out:
    pipeline.append(Article_to_Line(out, features))
    n_processed = process_xml(article_path, pipeline)
    print("Processed articles:", n_processed)


loading...
Processed articles: 645


In [0]:
pd.read_csv('data/train.text.tsv', sep='\t', header=None)

,0,1,2,3,4,5
0,0,True,None,None,From flickr.com : Money { MID-161793 } <splt> ...,Kucinich : Reclaiming the money power
1,1,True,None,None,Donald Trump ran on many braggadocios and larg...,Trump Just Woke Up & Viciously Attacked Puerto...
2,2,True,None,None,Photo By Justin Sullivan / Getty Images <splt>...,"Liberals wailing about gun control , but what ..."
3,3,True,None,None,After Colin Kaepernick rightly chose to kneel ...,Laremy Tunsil joins NFL players in kneeling du...
4,4,False,None,None,"Almost a half - century ago , in 1968 , the Un...",It 's 1968 All Over Again
...,...,...,...,...,...,...
640,640,True,None,None,Donald Trump . <splt> Photo from whitehouse.go...,Trump Turns his Back on American Workers
641,641,False,None,None,President Donald Trump on Tuesday began disman...,"Cummins : Rescinding DACA ‘ discriminatory , h..."
642,642,False,None,None,The US Supreme Court has ruled that Donald Tru...,"Trump travel ban can be enforced , says US Sup..."
643,643,False,None,None,"Ex - FBI Director James Comey went rogue , acc...",VIDEO- AG SESSIONS : Comey Went Rogue In Hilla...
